In [17]:
import pandas as pd
import numpy as np
import requests
import numpy as np
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = './Tesseract/Tesseract.exe'
import os

In [18]:
def tri_3pl_enem(theta, a, b, c):
    return c + (1 - c) / (1 + np.exp(-a * (theta - b)))

In [19]:
def find_quantile(c):
    return round(((1+c)/2)*100, 2)

In [20]:
def imageApi(code):
    code = str(str(code) + '.png')
    imagem = 'https://niedsonemanoel.com.br/enem/An%C3%A1lise%20de%20Itens/OrdenarPorTri/1.%20Itens%20BNI_/' + code
    return imagem

In [21]:
def get_prova_string(ano, co_prova, sg, cor):
    if ano == 2017:
      if co_prova in [201711,201710,201712,201709]:
          return str(str(sg)+' '+str(cor)+' REGULAR')
      else:
          return str(str(sg)+' '+str(cor)+' PPL')
    else: return str(str(sg)+' '+str(cor)+' REGULAR')

In [22]:
# Encontrando o valor de theta que resulta em targ
def find_theta(a, b, c, targ):
    left = -100
    right = 100
    tol = 1e-5
    if (targ == 0.6):
      target = find_quantile(c)/100
    else:
      target = targ
    while (right - left) / 2 > tol:
        theta = (left + right) / 2
        value = tri_3pl_enem(theta, a, b, c)
        if value > target:
            right = theta
        else:
            left = theta
    return theta * 100 + 500

In [ ]:
def ocrImage(code):
    output_file='../1. Itens BNI_/TXT/'+str(code)+'.txt'
    if os.path.exists(output_file):
        with open(output_file, 'r', encoding='utf-8') as file:
            return file.read()

    code = 'https://raw.githubusercontent.com/NiedsonEmanoel/NiedsonEmanoel/main/enem/An%C3%A1lise%20de%20Itens/OrdenarPorTri/1.%20Itens%20BNI_/'+str(str(code) + '.png')
    try:
        response = requests.get(code)
        img_array = np.array(bytearray(response.content), dtype=np.uint8)

        # Decodificar a imagem usando o OpenCV
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        ocrT = str(pytesseract.image_to_string(img, lang='por'))

        # Salvar o resultado no arquivo .txt se o arquivo não existir
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(ocrT)
            print(f"Arquivo '{output_file}' criado com sucesso!")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        ocrT = 'N/A'
    return ocrT

# Chamar a função para processar a imagem e salvar o resultado em um arquivo .txt se ele não existir
resultado_ocr = ocrImage(23448)
print(resultado_ocr)

In [23]:
def toENEM(dfItens, ano, provas):
    dfItens = dfItens[dfItens['TP_CERTIFICACAO'] == 2]
    dfItens = dfItens[dfItens.CO_PROVA.isin(provas)]

    dfItens['ANO'] = ano
    dfItens['IN_ENCCEJA'] = 1
    dfItens['IN_ENEM'] = 0
    dfItens['CO_PROVA'] = dfItens.apply(lambda row: get_prova_string(row['ANO'], row['CO_PROVA'], row['SG_AREA'], row['TX_COR'],), axis=1)
    
    for i in dfItens.index:
        dfItens.loc[i, "theta_065"] = find_theta(
            dfItens.loc[i, "NU_PARAM_A"],
            dfItens.loc[i, "NU_PARAM_B"],
            dfItens.loc[i, "NU_PARAM_C"],
            (find_quantile(dfItens.loc[i, "NU_PARAM_C"])/100),
        )
        dfItens.loc[i, "theta_080"] = find_theta(
            dfItens.loc[i, "NU_PARAM_A"],
            dfItens.loc[i, "NU_PARAM_B"],
            dfItens.loc[i, "NU_PARAM_C"],
            0.80,
        )
        dfItens.loc[i, "theta_099"] = find_theta(
            dfItens.loc[i, "NU_PARAM_A"],
            dfItens.loc[i, "NU_PARAM_B"],
            dfItens.loc[i, "NU_PARAM_C"],
            0.99,
        )
        dfItens.loc[i, 'PercentEspAcerto'] = find_quantile(
            dfItens.loc[i, "NU_PARAM_C"]
        )

    del dfItens['TP_CERTIFICACAO']

    dfItens.to_csv(str(ano)+'_ENCCEJA_ITENS_PROVA.csv', index=False, sep=";", encoding="latin-1")
    dfItens.to_excel('Itens Externos/ITENS_ENCCEJA/'+str(ano)+'_ENCCEJA_ITENS_PROVA.xlsx', index=False)
    return dfItens

In [24]:
toENEM(
    pd.read_csv('Itens Externos/ITENS_ENCCEJA/MICRODADOS_ENCCEJA_ITENS_PROVA.csv', sep=";", encoding="latin-1"),
    2017,
    [201711, 201710, 201712, 201709, 201723, 201722, 201724, 201721]
    )

pd2018 = pd.read_csv('Itens Externos/ITENS_ENCCEJA/ITENS_PROVA_2018.csv', sep=";", encoding="latin-1")
pd2019 = pd.read_csv('Itens Externos/ITENS_ENCCEJA/MICRODADOS_ENCCEJA_ITENS_PROVA_2019.csv', sep=";", encoding="latin-1")
pd2020 = pd.read_csv('Itens Externos/ITENS_ENCCEJA/MICRODADOS_ENCCEJA_ITENS_PROVA_2020.csv', sep=";", encoding="latin-1")